# Load data

In [1]:
import numpy as np 
from monty.json import MontyDecoder
from monty.serialization import loadfn

data = loadfn('bulk_moduli.json')
structures = data['structures']
targets = np.log10(data['bulk_moduli'])

# Set up model and train

In [3]:
from megnet.data.crystal import CrystalGraph
from megnet.data.graph import GaussianDistance
from megnet.models import MEGNetModel

model = MEGNetModel(10, 2, nblocks=1, lr=1e-2,
                    n1=4, n2=4, n3=4, npass=1, ntarget=1,
                    graph_converter=CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 10), 0.5)))

In [4]:
model.train(structures, targets, epochs=2)

Epoch 1/2
1/1 [==============================] - 5s 5s/step - loss: 4.0360
Epoch 2/2
1/1 [==============================] - 0s 168ms/step - loss: 3.6460


# Prediction

## Predict from structure

In [5]:
from mp_api.client import MPRester
# mpr = MPRester()
with MPRester() as mpr:

# let's get some structure
    structure = mpr.get_structure_by_material_id('mp-1143')

/home/shibuM/anaconda3/envs/megnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 5940.94it/s]


In [6]:
predicted_K = 10 ** model.predict_structure(structure).ravel()
print('The predicted K for {} is {} GPa'.format(structure.formula, predicted_K[0]))

The predicted K for Al4 O6 is 1.5347446203231812 GPa
